<img src="images/ragna-logo.png" width="200px" align="right"/>

# Basics of RAG-powered chat app

<hr>

## What is Retrieval-augmented generation (RAG)?

LLMs are trained on vast, but static datasets. This means, while they can predict answers for several general questions like:

<img src="images/chatgpt-what-is-pycon-us.png" width=60% style="border:1px solid black;"/>

They can't answer or hallucinate answers for recent events or specific topics:

<img src="images/chatgpt-when-is-pycon-us.png" width=50% style="border:1px solid black;"/>

**Retrieval-augmented generation (RAG)** is a method to augment foundational LLMs with **contextual data (documents)**, to reduce hallucinations and get around the limited space available in an LLM prompts (around 3,000 words for ChatGPT 3.5).

<img src="images/RAG-new.png" width=70%/>

## What is Ragna?

Open source library for RAG **Orchestration** with a Python API, REST API, and web UI.

It gives you a convenience tools to quickly build RAG workflows and applications, with any LLM or source storage you prefer.

<img src="images/ragna-architecture.png" width=80%/>

## Build a chat function with Ragna

### Step 0: Setup requirements

To use builtin LLMs like OpenAI, you will need API keys. For this tutorial, we have included a key for you.

In [ ]:
from pathlib import Path

from dotenv import load_dotenv

dotenv_path = Path.home() / Path("shared/pycon/.env")
assert load_dotenv(dotenv_path)

#### Side note: Local setup instructions 💻

On local computers, follow these step to get started with Ragna:

1. Install Ragna: `pip install 'ragna[all]'`
2. Run `ragna init` to create the `ragna.toml` config file with a guided CLI. 
3. [Get an OpenAI API key](https://platform.openai.com/api-keys) and set the relevant environment variable `export OPENAI_API_KEY="XXX"`

### Step 1: Select relevant documents

Let's use PyCon US 2024's [What is PyCon US?](https://us.pycon.org/2024/about/pycon/), and  [Onsite Information](https://us.pycon.org/2024/attend/onsite/) pages.

💡 **Tip:** There are more documents in the `/files` directory that you can explore.

In [ ]:
documents = [
    "files/what-is-pycon-us.pdf",
    "files/onsite-information.pdf",
]

### Step 2: Select assistants and source storage

🔗 [Check the available assistants in the docs](https://ragna.chat/en/stable/generated/tutorials/gallery_python_api/#step-3-select-an-assistant)

We are selecting OpenAI's GPT-3.5 and GPT-4 LLMs, and Chroma and LanceDB source storages.

In [ ]:
from ragna import Rag
from ragna.assistants import Gpt4, Gpt35Turbo16k
from ragna.source_storages import Chroma, LanceDB

In [ ]:
rag = Rag()

### Step 3: Start chat

Ragna is async by design.

In [ ]:
chat = rag.chat(
    documents=documents,
    source_storage=Chroma,
    assistant=Gpt4,
)

await chat.prepare()

### Step 4: Ask questions

In [ ]:
answer = await chat.answer("When is PyCon US 2024?")
print(f"\nLLM Response: \n\n{answer.content}\n")

Let's check the sources used:

In [ ]:
answer.sources

In [ ]:
for idx, source in enumerate(answer.sources, 1):
    print(f"{idx}.: {source.document.name}, page(s) {source.location}\n")
    print(source.content)
    print("#" * 80 + "\n")

#### Streaming answers

Ragna allows you to stream the answers, one chunk at a time, just set `stream=True`:

In [ ]:
answer = await chat.answer("What can I expect at PyCon US 2024?", stream=True)

print(f"\nLLM Response: \n\n")
      
async for chunk in answer:
    print(chunk, end= "")

#### Reducing hallucinations (errors)

Ragna tries to ensure only the sources are used for answering questions.

🔗 [For reference, see Ragna source code on GitHub highlighting the prompt.](https://github.com/Quansight/ragna/blob/3cef0f7da1f2ed90e5d0618bcad82f824d00dc5a/ragna/assistants/_openai.py#L25-L26)

In [ ]:
answer = await chat.answer("When is the PyLadies lunch?")
print(f"\nLLM Response: \n\n{answer.content}")

### Advanced configuration

`Rag().chat()` takes the following keyword arguments to help you optimize the quality of answers:

* `chunk_size` - Size of each chunk (sections of the document that contain context) to use.
* `chunk_overlap` - Size of the overlap with previous and next chunk for retrieving additional context for future prompts.
* `num_tokens` - Maximum number of context tokens, and in turn the number of document chunks, pulled out of the vector database.

You can also set these configurations in the web app (which we will see later).

<hr>

_❗️ **Warning:** Make sure to stop the Jupyter Kernel (in the JupyterLab Menu Bar, click on "Kernel" -> "Shut down Kernel") before proceeding to prevent the "insufficient VRAM" error._

<br>

**✨ Next: [Use Local LLM with Ragna](03-RAG-local-llm.ipynb) →**

<br>

💬 _Wish to continue discussions after the tutorial? Contact the presenters: [@pavithraes](https://github.com/pavithraes), [@dharhas](https://github.com/dharhas), [@ahuang11](https://github.com/ahuang11)_

<hr>